
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score
precision_score
recall_score

data = pd.read_csv('data/Corona_NLP_train.csv', encoding='Latin1')
data = data[data['Sentiment'] != 'Neutral']

X = data['OriginalTweet']
X = X.to_numpy()
Y = data['Sentiment']
Y = Y.apply(lambda x: 1 if x in['Positive', 'Extremely Positive'] else 0)
Y = Y.to_numpy()

n = len(Y)
train_end = int(0.6 * n)
test_end = train_end + int(0.15 * n)

Y_train, Y_test, Y_valid = Y[:train_end], Y[train_end:test_end], Y[test_end:]
X_train, X_test, X_valid = X[:train_end], X[train_end:test_end], X[test_end:]

FileNotFoundError: [Errno 2] No such file or directory: 'data/Corona_NLP_train.csv'

In [10]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

from sklearn.feature_extraction.text import CountVectorizer

model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(solver='liblinear'))
])

model.fit(X_train, Y_train)

probabilities = model.predict_proba(X_test)
print("Predicted Probabilities:", probabilities)

Predicted Probabilities: [[0.87765727 0.12234273]
 [0.99712784 0.00287216]
 [0.03466503 0.96533497]
 ...
 [0.5880236  0.4119764 ]
 [0.94842033 0.05157967]
 [0.92857117 0.07142883]]


In [11]:
# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

Y_pred = model.predict(X_test)

TP_1 = np.sum((Y_pred == 1) & (Y_test == 1))
FP_1 = np.sum((Y_pred == 1) & (Y_test == 0))
FN_1 = np.sum((Y_pred == 0) & (Y_test == 1))
TP_0 = np.sum((Y_pred == 0) & (Y_test == 0))
FP_0 = np.sum((Y_pred == 0) & (Y_test == 1))
FN_0 = np.sum((Y_pred == 1) & (Y_test == 0))

precision_0 = precision_score(Y_test, Y_pred)
precision_1 = TP_1 / (TP_1 + FP_1) if (TP_1 + FP_1) > 0 else 0
recall_0 = TP_0 / (TP_0 + FN_0) if (TP_0 + FN_0) > 0 else 0
recall_1 = recall_score(Y_test, Y_pred)

print(precision_1, recall_1)

0.8708971553610503 0.8808557727775729


In [12]:

# Part 4

 # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost

def cost(model,threshold,X,Y):

   probabilities = model.predict_proba(X)[:, 1] 
   predictions = (probabilities >= threshold).astype(int)
    
   # Initialize cost list
   costs = []
    
   for pred, true_label in zip(predictions, Y):
      if true_label == 1 and pred == 0:  # Positive classified as negative
            costs.append(1)
      elif true_label == 0 and pred == 1:  # Negative classified as positive
            costs.append(5)
      else:  # Correct classification
            costs.append(0)
    
   # Return the average cost
   return np.mean(costs)

In [13]:
# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

optimal_threshold = None
cost_at_optimal_threshold = float('inf')

for threshold in np.linspace(0, 1, 101): 
    current_cost = cost(model, threshold, X_test, Y_test)
    if current_cost < cost_at_optimal_threshold:
        cost_at_optimal_threshold = current_cost
        optimal_threshold = threshold

# Outputs
optimal_threshold, cost_at_optimal_threshold

(0.8, 0.2727272727272727)

In [20]:
# Part 6

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)
print(cost_at_optimal_threshold)
m = len(X_valid)
alpha = 0.01
delta = (1/np.sqrt(m))*np.sqrt((1/2)*np.log(2/alpha))
print("Confidence interval around the true cost [%.2f,%.2f]" % (cost_at_optimal_threshold-delta,cost_at_optimal_threshold+delta))
cost_interval_valid = (cost_at_optimal_threshold-delta, cost_at_optimal_threshold+delta)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

0.2727272727272727
Confidence interval around the true cost [0.25,0.29]


In [15]:

# Part 7

def random_cost(threshold, model, X, Y):
    probabilities = model.predict_proba(X)[:, 1]
    C = np.where(
        probabilities < threshold,  
        Y,                          
        5 * (1 - Y)              
    )
    return C

C = random_cost(optimal_threshold, model, X_valid, Y_valid)

m = len(C)
mean_C = np.mean(C)
var_C = np.var(C, ddof=0)  # Population variance (not sample variance)
    
M = np.max(C) - np.min(C)  # C ranges from 0 to 5, so M = 5
    
ln_term = np.log(2 / alpha)
epsilon = np.sqrt((2 * var_C * ln_term) / m) + (ln_term * M) / (3 * m)

lower_bound = mean_C - epsilon
upper_bound = mean_C + epsilon

CI = (lower_bound, upper_bound)

variance_of_C = var_C
interval_of_C = CI
print(interval_of_C, variance_of_C)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

(0.2472672792865777, 0.30762389507362314) 0.6692569047766288
